In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
#!apt update && apt install -y libsm6 libxext6

In [42]:
from pathlib import Path
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from salt_utils import *
from datasets import SaltDatasetArrays, get_dls
from unet import dpnunet68
from train import loss_batch, fit
from loss import lovasz_hinge

In [4]:
train_augs = get_tfms(128, val=False)
val_augs = get_tfms(128, val=True)

In [5]:
PATH = Path('/mnt/user-home/Data/salt/')
train_img_paths, train_mask_paths, val_img_paths, val_mask_paths = next(cross_val_paths(PATH))

In [6]:
#Need Data
trn_dl, val_dl = get_dls(train_img_paths, train_mask_paths, val_img_paths, val_mask_paths, pin_memory=False, batch_size=64, num_workers=12)

In [7]:
#We also need an arch
arch = dpnunet68()

In [8]:
from torch import nn
arch = arch.cuda()

In [35]:
x, y = next(iter(trn_dl))

In [36]:
y_ = arch(torch.autograd.Variable(x.permute(0,3,1,2)).cuda()) #hell yes it works.

In [37]:
#We need a loss function

In [38]:
loss = lovasz_hinge

In [39]:
loss(y_, y.cuda())

tensor(2.0191, device='cuda:0')

In [40]:
opt = torch.optim.SGD(arch.parameters(), lr=0.0001, momentum=0.9)

In [43]:
l, _, _ = loss_batch(arch, torch.autograd.Variable(x, requires_grad=True).cuda(), y.cuda(), loss, opt=opt)

In [44]:
fit(1, arch.cuda(), loss, opt, trn_dl, val_dl)

RuntimeError: Expected object of type torch.FloatTensor but found type torch.cuda.FloatTensor for argument #2 'weight'

In [16]:
l

2.044387102127075